# HW06: Деревья решений и ансамбли

## Задание

Цель: закрепить понимание деревьев решений и ансамблевых методов (bagging, random forest, boosting, stacking), а также провести честный ML-эксперимент.

In [ ]:
# Импорты
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import os

# Фиксируем случайный.seed для воспроизводимости
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2.3.1. Загрузка данных и первичный анализ

In [ ]:
# Загрузка данных
# Выберите один из датасетов: S06-hw-dataset-01.csv, S06-hw-dataset-02.csv, S06-hw-dataset-03.csv, S06-hw-dataset-04.csv
df = pd.read_csv('S06-hw-dataset-02.csv')  # Измените на нужный датасет

# Просмотр данных
print("Форма данных:", df.shape)
print("\nПервые строки:")
print(df.head())

print("\nИнформация о данных:")
print(df.info())

print("\nСтатистики:")
print(df.describe())

# Распределение таргета
print("\nРаспределение таргета:")
print(df['target'].value_counts(normalize=True))

In [ ]:
# Проверка пропусков и типов столбцов
print("Пропущенные значения:")
print(df.isnull().sum())

print("\nТипы столбцов:")
print(df.dtypes)

# Определение X и y
# Предполагаем, что столбец 'id' не используется как признак
if 'id' in df.columns:
    X = df.drop(['target', 'id'], axis=1)
else:
    X = df.drop(['target'], axis=1)
    
y = df['target']

print(f"Форма X: {X.shape}")
print(f"Форма y: {y.shape}")

## 2.3.2. Train/Test-сплит и воспроизводимость

In [ ]:
# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE, 
    stratify=y  # Для классификации используем стратификацию
)

print(f"Размер обучающей выборки: {X_train.shape[0]}")
print(f"Размер тестовой выборки: {X_test.shape[0]}")
print(f"Доли классов в train: {y_train.value_counts(normalize=True).to_dict()}")
print(f"Доли классов в test: {y_test.value_counts(normalize=True).to_dict()}")

# Пояснение важности фиксированного seed и стратификации
print("\nФиксированный seed обеспечивает воспроизводимость результатов.")
print("Стратификация сохраняет пропорции классов в train и test, что важно для корректной оценки модели.")

## 2.3.3. Baseline'ы

In [ ]:
# Baseline 1: DummyClassifier
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=RANDOM_STATE)
dummy_clf.fit(X_train, y_train)
y_pred_dummy = dummy_clf.predict(X_test)

print("Baseline 1 - DummyClassifier (most_frequent strategy):")
print(f"Accuracy: {accuracy_score(y_test, y_pred_dummy):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_dummy, average='macro'):.4f}")
if len(np.unique(y)) == 2:  # Если бинарная классификация
    print(f"ROC-AUC: {roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1]):.4f}")

# Baseline 2: Logistic Regression
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))
])

lr_pipeline.fit(X_train, y_train)
y_pred_lr = lr_pipeline.predict(X_test)

print("\nBaseline 2 - Logistic Regression:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_lr, average='macro'):.4f}")
if len(np.unique(y)) == 2:  # Если бинарная классификация
    y_pred_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]
    print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")

## 2.3.4. Модели недели 6

In [ ]:
# Модель 1: Decision Tree с контролем сложности
dt_params = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_leaf': [5, 10, 20]
}

dt_grid = GridSearchCV(
    DecisionTreeClassifier(random_state=RANDOM_STATE),
    dt_params,
    cv=5,
    scoring='roc_auc' if len(np.unique(y)) == 2 else 'f1_macro',
    n_jobs=-1
)

dt_grid.fit(X_train, y_train)
best_dt = dt_grid.best_estimator_

y_pred_dt = best_dt.predict(X_test)
if len(np.unique(y)) == 2:
    y_pred_proba_dt = best_dt.predict_proba(X_test)[:, 1]
    dt_roc_auc = roc_auc_score(y_test, y_pred_proba_dt)
else:
    dt_roc_auc = None

print("Decision Tree (best params):", dt_grid.best_params_)
print(f"Accuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_dt, average='macro'):.4f}")
if dt_roc_auc is not None:
    print(f"ROC-AUC: {dt_roc_auc:.4f}")

In [ ]:
# Модель 2: Random Forest
rf_params = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None],
    'min_samples_leaf': [5, 10]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    rf_params,
    cv=5,
    scoring='roc_auc' if len(np.unique(y)) == 2 else 'f1_macro',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)
best_rf = rf_grid.best_estimator_

y_pred_rf = best_rf.predict(X_test)
if len(np.unique(y)) == 2:
    y_pred_proba_rf = best_rf.predict_proba(X_test)[:, 1]
    rf_roc_auc = roc_auc_score(y_test, y_pred_proba_rf)
else:
    rf_roc_auc = None

print("Random Forest (best params):", rf_grid.best_params_)
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf, average='macro'):.4f}")
if rf_roc_auc is not None:
    print(f"ROC-AUC: {rf_roc_auc:.4f}")

In [ ]:
# Модель 3: Boosting (Gradient Boosting)
gb_params = {
    'n_estimators': [50, 100],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5]
}

gb_grid = GridSearchCV(
    GradientBoostingClassifier(random_state=RANDOM_STATE),
    gb_params,
    cv=5,
    scoring='roc_auc' if len(np.unique(y)) == 2 else 'f1_macro',
    n_jobs=-1
)

gb_grid.fit(X_train, y_train)
best_gb = gb_grid.best_estimator_

y_pred_gb = best_gb.predict(X_test)
if len(np.unique(y)) == 2:
    y_pred_proba_gb = best_gb.predict_proba(X_test)[:, 1]
    gb_roc_auc = roc_auc_score(y_test, y_pred_proba_gb)
else:
    gb_roc_auc = None

print("Gradient Boosting (best params):", gb_grid.best_params_)
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_gb, average='macro'):.4f}")
if gb_roc_auc is not None:
    print(f"ROC-AUC: {gb_roc_auc:.4f}")

In [ ]:
# Модель 4: Stacking (опционально)
base_models = [
    ('dt', DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE)),
    ('lr', Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(random_state=RANDOM_STATE, max_iter=1000))]))
]

stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(random_state=RANDOM_STATE),
    cv=5  # Кросс-валидация для получения прогнозов базовых моделей
)

stacking_clf.fit(X_train, y_train)
y_pred_stack = stacking_clf.predict(X_test)

if len(np.unique(y)) == 2:
    y_pred_proba_stack = stacking_clf.predict_proba(X_test)[:, 1]
    stack_roc_auc = roc_auc_score(y_test, y_pred_proba_stack)
else:
    stack_roc_auc = None

print("Stacking Classifier:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_stack):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_stack, average='macro'):.4f}")
if stack_roc_auc is not None:
    print(f"ROC-AUC: {stack_roc_auc:.4f}")

## 2.3.5. Метрики качества

In [ ]:
# Сравнение всех моделей по всем метрикам
models_results = {}

# Словарь моделей
all_models = {
    'Dummy': (dummy_clf, y_pred_dummy, None if len(np.unique(y)) != 2 else dummy_clf.predict_proba(X_test)[:, 1]),
    'Logistic_Regression': (lr_pipeline, y_pred_lr, y_pred_proba_lr if len(np.unique(y)) == 2 else None),
    'Decision_Tree': (best_dt, y_pred_dt, y_pred_proba_dt if len(np.unique(y)) == 2 else None),
    'Random_Forest': (best_rf, y_pred_rf, y_pred_proba_rf if len(np.unique(y)) == 2 else None),
    'Gradient_Boosting': (best_gb, y_pred_gb, y_pred_proba_gb if len(np.unique(y)) == 2 else None),
    'Stacking': (stacking_clf, y_pred_stack, y_pred_proba_stack if len(np.unique(y)) == 2 else None)
}

# Вычисляем метрики для всех моделей
for name, (model, y_pred, y_proba) in all_models.items():
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    
    if y_proba is not None:
        roc_auc = roc_auc_score(y_test, y_proba)
    else:
        roc_auc = None
    
    models_results[name] = {
        'accuracy': acc,
        'f1_score': f1,
        'roc_auc': roc_auc
    }

# Вывод таблицы результатов
results_df = pd.DataFrame(models_results).T
print(results_df)

In [ ]:
# Графики
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Confusion Matrix для лучшей модели (предположим, это Random Forest)
best_model_name = 'Random_Forest'  # Можно определить программно какую модель выбрать как лучшую
best_y_pred = all_models[best_model_name][1]

cm = confusion_matrix(y_test, best_y_pred)
sns.heatmap(cm, annot=True, fmt='d', ax=axes[0,0])
axes[0,0].set_title(f'Confusion Matrix - {best_model_name}')
axes[0,0].set_xlabel('Predicted')
axes[0,0].set_ylabel('Actual')

# ROC Curve для бинарной классификации
if len(np.unique(y)) == 2:
    from sklearn.metrics import roc_curve
    
    fpr, tpr, _ = roc_curve(y_test, all_models[best_model_name][2])
    axes[0,1].plot(fpr, tpr, label=f'{best_model_name} (AUC = {models_results[best_model_name]["roc_auc"]:.3f})')
    axes[0,1].plot([0, 1], [0, 1], 'k--', label='Random')
    axes[0,1].set_xlabel('False Positive Rate')
    axes[0,1].set_ylabel('True Positive Rate')
    axes[0,1].set_title('ROC Curve')
    axes[0,1].legend()
else:
    axes[0,1].text(0.5, 0.5, 'ROC-AUC не применима\\nк мультиклассу', ha='center', va='center', transform=axes[0,1].transAxes)
    axes[0,1].set_title('ROC Curve (not applicable for multiclass)')

# Сравнение метрик
metrics_comparison = results_df[['accuracy', 'f1_score']].dropna(axis=1, how='all')
metrics_comparison.plot(kind='bar', ax=axes[1,0])
axes[1,0].set_title('Сравнение Accuracy и F1 Score')
axes[1,0].set_xlabel('Модель')
axes[1,0].set_ylabel('Метрика')
axes[1,0].tick_params(axis='x', rotation=45)

# ROC-AUC если применимо
if 'roc_auc' in results_df.columns and not results_df['roc_auc'].isna().all():
    results_df['roc_auc'].dropna().plot(kind='bar', ax=axes[1,1])
    axes[1,1].set_title('ROC-AUC')
    axes[1,1].set_xlabel('Модель')
    axes[1,1].set_ylabel('ROC-AUC')
    axes[1,1].tick_params(axis='x', rotation=45)
else:
    axes[1,1].text(0.5, 0.5, 'ROC-AUC не доступна', ha='center', va='center', transform=axes[1,1].transAxes)
    axes[1,1].set_title('ROC-AUC')

plt.tight_layout()
plt.savefig('artifacts/figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 2.3.6. Интерпретация

In [ ]:
# Определяем лучшую модель (например, по ROC-AUC для бинарной или F1 для мультикласса)
if len(np.unique(y)) == 2:
    best_model_key = max(models_results.keys(), key=lambda x: models_results[x]['roc_auc'] if models_results[x]['roc_auc'] is not None else -float('inf'))
else:
    best_model_key = max(models_results.keys(), key=lambda x: models_results[x]['f1_score'])

best_model = all_models[best_model_key][0]
print(f"Лучшая модель: {best_model_key}")

# Permutation Importance
perm_importance = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, n_jobs=-1)

# Получаем топ-10 признаков
feature_names = X.columns
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance_mean': perm_importance.importances_mean,
    'importance_std': perm_importance.importances_std
}).sort_values(by='importance_mean', ascending=False)

top_features = importance_df.head(10)
print("\nТоп-10 наиболее важных признаков:")
print(top_features)

# Визуализация permutation importance
plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['importance_mean'], xerr=top_features['importance_std'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Permutation Importance')
plt.title(f'Top-10 Feature Importances ({best_model_key})')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('artifacts/figures/permutation_importance.png', dpi=300, bbox_inches='tight')
plt.show()

## 2.4. Сохранение артефактов

In [ ]:
# Сохраняем метрики на тесте
with open('artifacts/metrics_test.json', 'w') as f:
    json.dump({k: {metric: float(v) if isinstance(v, (np.float64, np.float32)) else v for metric, v in v.items()} for k, v in models_results.items()}, f, indent=2)

# Сохраняем результаты подбора гиперпараметров
search_summaries = {
    'DecisionTree': {
        'best_params': dt_grid.best_params_,
        'cv_score': float(dt_grid.best_score_),
        'model_type': 'DecisionTree'
    },
    'RandomForest': {
        'best_params': rf_grid.best_params_,
        'cv_score': float(rf_grid.best_score_),
        'model_type': 'RandomForest'
    },
    'GradientBoosting': {
        'best_params': gb_grid.best_params_,
        'cv_score': float(gb_grid.best_score_),
        'model_type': 'GradientBoosting'
    }
}

with open('artifacts/search_summaries.json', 'w') as f:
    json.dump(search_summaries, f, indent=2)

# Сохраняем лучшую модель
joblib.dump(all_models[best_model_key][0], 'artifacts/best_model.joblib')

# Сохраняем метаданные лучшей модели
best_model_meta = {
    'model_name': best_model_key,
    'model_type': str(type(all_models[best_model_key][0])),
    'test_metrics': {
        'accuracy': float(models_results[best_model_key]['accuracy']),
        'f1_score': float(models_results[best_model_key]['f1_score']),
        'roc_auc': float(models_results[best_model_key]['roc_auc']) if models_results[best_model_key]['roc_auc'] is not None else None
    },
    'cv_score': getattr(eval(f'{best_model_key.lower()}_grid'), 'best_score_', None),
    'best_params': getattr(eval(f'{best_model_key.lower()}_grid'), 'best_params_', None) if best_model_key in ['Decision_Tree', 'Random_Forest', 'Gradient_Boosting'] else 'N/A'
}

with open('artifacts/best_model_meta.json', 'w') as f:
    json.dump(best_model_meta, f, indent=2)

print("Артефакты сохранены успешно!")